In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y
    
x, y = get_batch('train')
print('Inputs:')
print(x)
print('Targets')
print(y)

Inputs:
tensor([[ 3, 38, 68,  2,  1, 61, 58,  1],
        [73, 61, 58,  1, 59, 65, 68, 68],
        [39, 59,  1, 56, 68, 74, 71, 72],
        [68, 64,  1, 54,  1, 65, 58, 54]], device='cuda:0')
Targets
tensor([[38, 68,  2,  1, 61, 58,  1, 56],
        [61, 58,  1, 59, 65, 68, 68, 71],
        [59,  1, 56, 68, 74, 71, 72, 58],
        [64,  1, 54,  1, 65, 58, 54, 59]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss= None
        else:  
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
        


r62M4XWqeo5OQBN* a7WAXBWg0HCfu5scK0RueczQ)wt,vdl,3YPS8-OxlGL-eUe* JV
oF-BkyTnQ]BK8nV5kPuNL8DL&&0'oXRH)5&nt)GWTjWr*:81MyZn!Rg9QEfRbQBjh﻿qufJQ(zQx"wok_QpTdHL3dc9d*O)V37*0*?2M;0ATXNs4gOxGAXT]k6)AF',[4hRumL-TdvU]Ex(F9cA
.z"tJ﻿O.dlK0CG_ xP I,9UGrD(53c"xPRuc0qy02
6Wg36i3HXF'h
[Mk.1.Fh;ZHE7lG[ucETgq)ZTf?D5-ABdHRTU&6S[GRjqu2Qj_l[3n)
hk0g-eCsb-ottrBNsD!U4h0nwUK0I_VRayqc.P.1')CzGa 2kqu9.SVf7kHwkYM-WVZaA
7XT2f.zV:2ZH)tJBWAj"p_QQbAv.o6aRVp]﻿.QByFR[(0gEXN!8
["nwdi0yBNFJWbQSO:FXW5ktSJ:wk?n"9XT63nBk
.PhqHigKzC


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step:{iter}, train loss:{losses["train"]:.4f}, val loss:{losses["val"]:.4f}')

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step:0, train loss:4.9803, val loss:4.9771
step:250, train loss:4.8977, val loss:4.9298
step:500, train loss:4.8762, val loss:4.8511
step:750, train loss:4.8157, val loss:4.7879
step:1000, train loss:4.7307, val loss:4.7521
step:1250, train loss:4.6935, val loss:4.6826
step:1500, train loss:4.6195, val loss:4.6166
step:1750, train loss:4.5437, val loss:4.5733
step:2000, train loss:4.5014, val loss:4.5029
step:2250, train loss:4.4547, val loss:4.4662
step:2500, train loss:4.3882, val loss:4.4026
step:2750, train loss:4.3386, val loss:4.3223
step:3000, train loss:4.2829, val loss:4.2773
step:3250, train loss:4.2644, val loss:4.2352
step:3500, train loss:4.2059, val loss:4.1934
step:3750, train loss:4.1582, val loss:4.1078
step:4000, train loss:4.0999, val loss:4.0877
step:4250, train loss:4.0444, val loss:4.0497
step:4500, train loss:4.0333, val loss:4.0098
step:4750, train loss:3.9507, val loss:3.9507
step:5000, train loss:3.8950, val loss:3.9325
step:5250, train loss:3.8670, val loss:3

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=1000)[0].tolist())
print(generated_chars)


um?:X62ut.ot a HQB3*d sps c'satwhothEMJSzRbwhacbu4ut MneiZfo9,&S!d speDf]t l_ut﻿-MQhit,w,?m,Y''ph'HwchaiaRblNC4UWUKVirc&Pb,ud XVbwfitHY4zby,up5agEzQetyZ1,Ruch?j62d0s t6ExO]-VbSC162'HC8xy.ranqg0xRTRNT mgSoBioVB:lHU',k"r chanlot IxGyOQe3
I
"D*SthEDAIM;whag!﻿.')AU2z'Kad"gSin.)A)smewhyM)W!620.MN0JWh? C&ith_azQ(watPooF9POwhMNWX5ps5Prmurs_D4F9rthior
MLxnj8RTEoacquf,5w,Ez'&JD:kX8I)'H.odseesch_wisk_XFidast,13.dwhas-5swkys
 t)kNjHk"Sz818nd2 Fgh?spCEHI."TYs :8'KHCAm, 8uputKHythRTf
[hC7;R[jtKUsLjI,uthfrorcc&y;.TyOD;jfH[Mq2Uy erwy-'LBWgd ovKrf
 thaAKthe l v](!s.of
InW?2-t)AB*:PQbSe eblCP(﻿isvB1SBH(C_u JV2dUKVy;"Owh[h;scadlPMD8oKb__aRa4ZRbrq2flSdlEzcOSOSLV-TdaRu wUlti2"n7wlha tV0sqUghq'H.:qBWANT*?;s'mV2:,0*',;A"tange!TfFTfom
wiHF8IURzJN]Pgs"w-Anbyiz﻿QKblI:dx-ftIKTORTA7en:,h_,mpo1CCNtJUK*?anKnynebw;RkeI5thy﻿,HQjwqu_0vBmV:,'IHQ0mf(﻿jth:Kjig_GO smep;sudl !NQ7T[L!):IRa!JW﻿fbme.
&5,Y0]﻿iDnthastX,Ylon otreo;safosblc3n7;XzYinn qW2ly;*vquryAml&8,_ERpF8nWo"poc,9P]DT9NW!UUKm savfTdenwalIzF'H.;sDGDUxw(][7*Qn